In [147]:
#IMPORT PACKAGES

import numpy as np
import math
from collections import Counter
import requests

%matplotlib inline
import matplotlib.pyplot as pl
import matplotlib.pylab as pylab
from matplotlib.patches import Polygon

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import cufflinks as cf
from IPython.display import display, HTML


import seaborn
import pandas as pd

from simple_salesforce import Salesforce

In [148]:
#CLASS THAT ORGANIZES QUERIED DATA INTO A PANDAS DATAFRAME

class Opportunities_DataFrame(object):
    def __init__(self, APIkey):
        self.APIkey = APIkey
        
    
    def querySalesforce(self):

        with open(self.APIkey) as file:
            username, password, token = [x.strip("\n") for x in file.readlines()]
        sf = Salesforce(username=username, password=password, security_token=token)
        self.opportunities = sf.query("SELECT Opportunity.Account.MasterRecordID, Opportunity.Account.Name, Group_ID__c, Name, Opportunity.Account.Industry__c, CloseDate, Month__c, Amount, Skier_Count__c, Skier_Days__c, Lessons__c, Rentals__c, No_of_Days__c, Transportation__c, Opportunity.Account.BillingStreet, Opportunity.Account.BillingCity, Opportunity.Account.BillingPostalCode, Opportunity.Account.BillingState, Date_Booked__c, (SELECT Contact.Name,Contact.Email, Contact.Id, Contact.MailingStreet, Contact.MailingCity, Contact.MailingPostalCode, Contact.MailingState, OpportunityId, Role from OpportunityContactRoles where IsPrimary=True) FROM Opportunity")   

        #Extract Account ID
        self.Account_IDs = [x['Account']['MasterRecordId'] for x in self.opportunities["records"]]
        self.Account_IDs_counts = Counter(self.Account_IDs)

        #Extract Account Name
        self.Account_names = [x['Account']['Name'] for x in self.opportunities["records"]]
        self.Account_names_counts = Counter(self.Account_names)

        #Extract Group IDs
        self.Group_IDs = [x['Group_ID__c'] for x in self.opportunities["records"]]
        self.Group_IDs_counts = Counter(self.Group_IDs)

        #Extract Opportunity/Group Names
        self.Group_names = [x['Name'] for x in self.opportunities["records"]]
        self.Group_names_counts = Counter(self.Group_names)

        #Extract Industry Type
        self.Industries = [x['Account']['Industry__c'] for x in self.opportunities["records"]]
        self.Industries_counts = Counter(self.Industries)

        #Extract 1st Ski Date
        self.First_day = [x['CloseDate'] for x in self.opportunities["records"]]
        self.First_day_counts = Counter(self.First_day)

        #Extract Date Booked
        self.Date_Booked = [x['Date_Booked__c'] for x in self.opportunities["records"]]
        self.Date_Booked_counts = Counter(self.Date_Booked)

        #Extract Visit Period Month
        self.Months = [x['Month__c'] for x in self.opportunities["records"]]
        self.Months_counts = Counter(self.Months)

        #Extract Group Revenue
        self.Revenue = [x['Amount'] for x in self.opportunities["records"]]
        self.Revenue_counts = Counter(self.Revenue)

        #Extract Skier Count
        self.Skier_number = [x['Skier_Count__c'] for x in self.opportunities["records"]]
        self.Skier_number_counts = Counter(self.Skier_number)

        #Extract Skier Days
        self.Skier_days = [x['Skier_Days__c'] for x in self.opportunities["records"]]
        self.Skier_days_counts = Counter(self.Skier_days)

        #Extract Lessons Binary: Yes or No (1 or 0)
        self.Lessons = [x['Lessons__c'] for x in self.opportunities["records"]]
        self.Lessons_counts = Counter(self.Lessons)

        #Extract Rentals Binary: Yes or No (1 or 0)
        self.Rentals = [x['Rentals__c'] for x in self.opportunities["records"]]
        self.Rentals_counts = Counter(self.Rentals)

        #Extract Visit Duration (Number of Days)
        self.Duration = [x['No_of_Days__c'] for x in self.opportunities["records"]]
        self.Duration_counts = Counter(self.Duration)

        #Extract Transportation Mode (Cars, Bus, Vans)
        self.Transportation = [x['Transportation__c'] for x in self.opportunities["records"]]
        self.Duration_counts = Counter(self.Duration)

        #Extract Billing Address
        self.Address = [x['Account']['BillingStreet'] for x in self.opportunities["records"]]
        self.Address_counts = Counter(self.Address)

        #Extract Billing City
        self.Cities = [x['Account']['BillingCity'] for x in self.opportunities["records"]]
        self.Cities_counts = Counter(self.Cities)

        #Extract Billing Zip Code
        self.Zipcodes = [x['Account']['BillingPostalCode'] for x in self.opportunities["records"]]
        self.Zipcodes_counts = Counter(self.Zipcodes)

        #Extract Billing State
        self.States = [x['Account']['BillingState'] for x in self.opportunities["records"]]
        self.States_counts = Counter(self.States)
        
        #Extract Group Leader Name
        self.Groupleaders = []
        for x in self.opportunities["records"]:
            if x["OpportunityContactRoles"] is not None:
                self.Groupleaders.append(x["OpportunityContactRoles"]["records"][0]["Contact"]["Name"])
        else:
            self.Groupleaders.append('None')

        #Extract Group Leader Mailing Street
        self.GL_streets = []
        for x in self.opportunities["records"]:
            if x["OpportunityContactRoles"] is not None:
                self.GL_streets.append(x["OpportunityContactRoles"]["records"][0]["Contact"]["MailingStreet"])
            else:
                self.GL_streets.append('None')

        #Extract Group Leader Mailing City
        self.GL_cities = []
        for x in self.opportunities["records"]:
            if x["OpportunityContactRoles"] is not None:
                self.GL_cities.append(x["OpportunityContactRoles"]["records"][0]["Contact"]["MailingCity"])
        else:
            self.GL_cities.append('None')

        #Extract Group Leader Mailing Postal Code
        self.GL_zipcode = []
        for x in self.opportunities["records"]:
            if x["OpportunityContactRoles"] is not None:
                self.GL_zipcode.append(x["OpportunityContactRoles"]["records"][0]["Contact"]["MailingPostalCode"])
        else:
            self.GL_zipcode.append('None')

        #Extract Group Leader Mailing State
        self.GL_states = []
        for x in self.opportunities["records"]:
            if x["OpportunityContactRoles"] is not None:
                self.GL_states.append(x["OpportunityContactRoles"]["records"][0]["Contact"]["MailingState"])
            else:
                self.GL_states.append('None')

#FINISH BUILDING DATAFRAME

    def buildDataFrame(self):
    
        self.Opportunities_df = pd.DataFrame({'AccountID':self.Account_IDs, 'AccountName': self.Account_names, 
                                              'GroupID':self.Group_IDs, 'GroupName': self.Group_names, 
                                              #'GroupLeader':self.Groupleaders, 'LeaderStreet':self.GL_streets,
                                              #'LeaderCity': self.GL_cities, 'LeaderState': self.GL_states,
                                              #'LeaderZipcode': self.GL_zipcode, 
                                              'Industry':self.Industries,
                                              'ArrivalDate':self.First_day, 'Month': self.Months, 'Revenue': self.Revenue,
                                              'SkierCount':self.Skier_number, 'SkierDays': self.Skier_days,
                                              'Lesson':self.Lessons, 'Rentals':self.Rentals, 'Duration':self.Duration,
                                              'Transportation':self.Transportation, 'Address':self.Address,
                                              'City':self.Cities, 'Zipcode': self.Zipcodes, 'State':self.States,
                                              'DateBooked': self.Date_Booked})
 

In [149]:
#FUNCTIONS FOR CLEANING AND WRANGLING DATA

def spliceYears(data):
    #Data Cleaning
    clean_opps = data
    clean_opps['ArrivalDate'] = pd.to_datetime(clean_opps['ArrivalDate'])
    clean_opps['DateBooked'] = pd.to_datetime(clean_opps['DateBooked'])
    clean_opps['VisitCount_AllTIme']= clean_opps.groupby(['AccountName'])['ArrivalDate'].transform('count')
    #num_records = len(clean_opps)
    clean_opps.to_csv('/Users/AnthonyAbercrombie/projects/Monarch_Salesforce/clean_opps.csv', encoding='utf-8')
    
    #Splice dataframe for 2015-2016 Ski Season and clean_opps_2014-2015
    clean_opps = clean_opps.sort_values(by='ArrivalDate',ascending=False)
    clean_opps_dateindex = clean_opps.set_index('ArrivalDate')
    clean_opps_2015to2016 = clean_opps_dateindex['2016-04-10':'2015-11-26']
    clean_opps_2014to2015 = clean_opps_dateindex['2015-04-10':'2014-11-01']

    #Sort by Account Name so we can clusters of multi-visit accounts together.
    clean_opps_2015to2016_acct_cluster = clean_opps_2015to2016.sort_values(by='AccountName')
    clean_opps_2014to2015_acct_cluster = clean_opps_2014to2015.sort_values(by='AccountName')

    #Reset Index for easier data analysis
    clean_opps_2015to2016_acct_cluster_noindex = clean_opps_2015to2016_acct_cluster.reset_index()
    clean_opps_2014to2015_acct_cluster_noindex = clean_opps_2014to2015_acct_cluster.reset_index()

    #View for easily printing address labels and Group Leader names, which are used for mailing purposes. Will merge with Account_names_counts_df
    GL_addresses = clean_opps_2015to2016_acct_cluster_noindex[['AccountName', 'GroupName','ArrivalDate','Industry','Address','City','Zipcode','State','DateBooked']]

    #Add column that describes the number of times a group account has visited Monarch in a season.
    GL_addresses['VisitCount_2015to2016']=GL_addresses.groupby(['AccountName'])['ArrivalDate'].transform('count')
    clean_opps_2015to2016_acct_cluster_noindex['VisitCount_2015to2016']= clean_opps_2015to2016_acct_cluster_noindex.groupby(['AccountName'])['City'].transform('count')
    clean_opps_2014to2015_acct_cluster_noindex['VisitCount_2014to2015']= clean_opps_2014to2015_acct_cluster_noindex.groupby(['AccountName'])['City'].transform('count')

    #write to CSV
    GL_addresses.to_csv('/Users/AnthonyAbercrombie/projects/Monarch_Salesforce/labels4.csv',encoding='utf-8')
    clean_opps_2015to2016 = clean_opps_2015to2016_acct_cluster_noindex
    clean_opps_2014to2015 = clean_opps_2014to2015_acct_cluster_noindex
    
    return clean_opps, clean_opps_2015to2016, clean_opps_2014to2015

def valueCount(data, parameter):
    #Number of distinct values for a given parameter
    value_count = pd.value_counts(data[parameter].values, sort = False)
    return value_count

def groupVisits(data, suffix):
    suffix = suffix
    visit_column = 'VisitCount_%s' % suffix
    Groupnames = data[['AccountName', 'GroupName', visit_column]]
    visits = Groupnames.drop_duplicates(subset='AccountName')[visit_column].tolist()
    return visits
    

def list_of_Column_Values(data, parameter):
    list = data[parameter].tolist()
    return list

def RepeatGroups(data_currentyear, data_previousyear):
    RepeatGroups_Instances = [['AccountName', 'Revenue','SkierCount','Duration','SkierDays','VisitCount_2015to2016']]
    for index, row in data_currentyear.iterrows():
        if row['VisitCount_AllTIme' ] != row['VisitCount_2015to2016']:
            RepeatGroups_Instances.append([row['AccountName'], row['Revenue'], row['SkierCount'], row['Duration'], row['SkierDays'],
                                                     row['VisitCount_2015to2016']])
        else:
            pass

    #RepeatGroups = list(set(RepeatGroups_Instances_2014to2016))
    headers = RepeatGroups_Instances.pop(0)
    RepeatGroups_Instances_df = pd.DataFrame(RepeatGroups_Instances, columns=headers)

    RepeatGroups_Instances_df = pd.merge(RepeatGroups_Instances_df, data_previousyear[['AccountName','VisitCount_2014to2015','VisitCount_AllTIme']], on= 'AccountName',how='left').dropna()
    RepeatGroups_Instances_df = RepeatGroups_Instances_df.drop_duplicates()

    #RepeatGroups_Instances_grouped = RepeatGroups_Instances_2014to2016_df.groupby('AccountName').sum()
    RepeatGroups_Instances_grouped = RepeatGroups_Instances_df.groupby(['AccountName','VisitCount_2015to2016','VisitCount_2014to2015','VisitCount_AllTIme']).agg({'Revenue': np.sum,'SkierCount': np.sum, 'Duration': np.sum, 'SkierDays':np.sum})
    #RepeatGroups_Instances_grouped['VisitCount_2015to2016'] = RepeatGroups_Instances_grouped['VisitCount_2015to2016'].apply(np.sqrt)
    #RepeatGroups_Instances_grouped['VisitCount_2014to2015'] = RepeatGroups_Instances_grouped['VisitCount_2014to2015'].apply(np.sqrt)
    
    return RepeatGroups_Instances_df, RepeatGroups_Instances_grouped

def sum_Column(data, parameter):
    stat_sum = data[parameter].sum()
    return stat_sum

def mean_Column(data, parameter):
    stat_mean = data[parameter].mean()
    return stat_mean

def median_Column(data, parameter):
    stat_median = data[parameter].median()
    return stat_median

def sum_adjusted_SkierDays(data):
    stat_sum = (data['SkierDays']/data['Duration']).sum()
    return stat_sum
    
def mean_adjusted_SkierDays(data):
    stat_mean = (data['SkierDays']/data['Duration']).mean()
    return stat_mean

def median_adjusted_SkierDays(data):
    stat_median = (data['SkierDays']/data['Duration']).median()
    return stat_median

def topAccounts(data):
    Opps_by_Rev = data.sort_values(by = 'Revenue', ascending=False)

    Magic80_Accounts = Opps_by_Rev.iloc[0:275].groupby(['AccountName', 'Industry','Address','Zipcode','State','VisitCount_AllTIme']).agg({'Revenue': np.sum,'SkierCount': np.sum, 'Duration': np.sum, 'SkierDays':np.sum})

    Magic80_sorted = Magic80_Accounts.sort_values(by = 'Revenue', ascending = False)

    Magic80_Accounts_Revenue = Magic80_Accounts['Revenue'].reset_index().groupby('AccountName').agg({'Revenue': np.sum}).reset_index()
    Magic80_Accounts_SkierDays = Magic80_Accounts['SkierDays'].reset_index().groupby('AccountName').agg({'SkierDays': np.sum}).reset_index()

    Over_10k_Groups = Magic80_Accounts_Revenue[Magic80_Accounts_Revenue.Revenue > 10000].sort_values(by='Revenue', ascending = 'False')
    Over_10k_Groups2 = pd.merge(Over_10k_Groups, Magic80_Accounts_SkierDays, on = 'AccountName', how= 'inner').sort_values(by = 'Revenue', ascending = False)
    
    return Over_10k_Groups2

def category_stats(data, parameter): 
    category = data.groupby(parameter).sum()
    category = category.reset_index()
    return category

def generate_html_table(data):
    html_table = data.to_html(classes='table', index = False, escape = False)
    return html_table

def timeseries(data):
    timeseries = data.groupby('ArrivalDate').sum()
    timeseries = timeseries.reset_index()
    return timeseries

In [150]:
#FUNCTIONS FOR CREATING GRAPHS AND FIGURES


def twoYearOverview():
    #Plot of 2-year overview comparision
    trace0 = go.Bar(
        x=[np.nansum(Skierdays_2014to2015),np.nansum(Skierdays_2015to2016)],
        y= ['2014 - 2015 \n Skier Days','2015 - 2016 \n Skier Days'],
        marker=dict(
            color='rgba(144,87,180,0.6)',
            line=dict(
            color='rgba(144,87,180,1.0)',
            width=0.5,
            ),
        ),
        name='Skier Days',
        orientation='h',
    )

    trace1 = go.Bar(
        x=[np.nansum(Revenue_2014to2015),np.nansum(Revenue_2015to2016)],
        y=['2014 - 2015 \n Revenue', '2015 - 2016 \n Revenue'],
        marker=dict(
            color = 'rgba(87,180,144,0.6)',
            line = dict(
                color='rgba(87,180,144,1.0)',
                width=0.5,
            ),
        ),
        name = 'Gross Revenue',
        orientation='h'
    )

    trace2 = go.Bar(
        x=[np.nansum(Visits_2014to2015),np.nansum(Visits_2015to2016), len(RepeatGroups_Instances_df)],
        y=['2014 - 2015 \n Number of Visits', '2015 - 2016 \n Number of Visits', 'Visits from Repeat Groups'],
        marker=dict(
            color = 'rgba(245,24,86,0.6)',
            line = dict(
                color='rgba(245,24,86,1.0)',
                width=0.5,
            ),
        ),
        name = 'Number of Group Visits',
        orientation='h'
    )

    trace3 = go.Bar(
        x= [len(RepeatGroups_Instances_grouped),len(Visits_2015to2016), len(Visits_2014to2015)],
        y= ['Repeat Groups \n From Both Years', '# of Groups 2015-2016', '# of Groups 2014-2015'],
        marker=dict(
            color = 'rgba(255,200,87,0.6)',
            line = dict(
                color='rgba(255,200,87,1.0)',
                width=0.5,
            ),
        ),
        name = 'Distinct Groups (Accounts)',
        orientation='h'
    )

    layout = dict(
        title="Comparing the 2015-2016 Ski Season to 2014-2015",
        yaxis1 = dict(
            showgrid=False,
            showline=True,
            showticklabels=True,
            #domain=[.6, 0.69],
        ),
        yaxis2 = dict(
            showgrid=False,
            showline=True,
            showticklabels=True,
            #domain=[0.7, 0.79],
        ),
        yaxis3 = dict(
            showgrid=False,
            showline=True,
            showticklabels=True,
            #domain=[0.8, 0.89],
        ),
        yaxis4 = dict(
            showgrid=False,
            showline=True,
            showticklabels=True,
            #domain=[0.9, 1],
        ),
        xaxis1=dict(
            zeroline=False,
            showline=False,
            showticklabels=False,
            showgrid=True,
            domain=[0,0.85]
        ),
        xaxis2=dict(
            zeroline=False,
            showline=False,
            showticklabels=False,
            showgrid=True,
            domain=[0,0.85]
        ),
        xaxis3=dict(
            zeroline=False,
            showline=False,
            showticklabels=False,
            showgrid=True,
            domain=[0,0.85]
        ),
        xaxis4=dict(
            zeroline=False,
            showline=False,
            showticklabels=False,
            showgrid=True,
            domain=[0,0.85]
        ),
        legend=dict(
            x=0.85,
            y=0.5,
            font=dict(
                size=12,
            ),
        ),
        margin=dict(
            l=175,
            r=150,
            t=70,
            b=20,
        ),
        width=800,
        height=500,
        paper_bgcolor='rgb(248, 248, 255)',
        plot_bgcolor='rgb(248, 248, 255)',
    )

    annotations = []

    y_skierdays = np.rint([np.nansum(Skierdays_2014to2015),np.nansum(Skierdays_2015to2016)])
    skier_labels = ['2014 - 2015 \n Skier Days','2015 - 2016 \n Skier Days']
    y_revenue = np.round([np.nansum(Revenue_2014to2015),np.nansum(Revenue_2015to2016)], decimals=2)
    rev_labels = ['2014 - 2015 \n Revenue','2015 - 2016 \n Revenue']
    y_visits = [np.nansum(Visits_2014to2015),np.nansum(Visits_2015to2016), len(RepeatGroups_Instances_df)]
    vis_labels = ['2014 - 2015 \n Number of Visits', '2015 - 2016 \n Number of Visits', 'Visits from Repeat Groups']
    y_repeats = [len(RepeatGroups_Instances_grouped),len(Visits_2015to2016), len(Visits_2014to2015)]
    rep_labels = ['Repeat Groups \n From Both Years', '# of Groups 2015-2016', '# of Groups 2014-2015']

    #for yski, xskid, yrev, xrevd, yvis, xvisd, yrep, xrepd in zip(
       # y_skierdays, skier_labels, y_revenue, rev_labels,
        #y_visits, vis_labels,y_repeats, rep_labels):

    for yski, xskid in zip(y_skierdays, skier_labels):
        annotations.append(dict(xref='x1', yref='y1', y = xskid, x = yski,
                                xanchor = 'right',
                                text='{:,}'.format(yski) + ' Skiers Days',
                                font = dict(family='Arial',size=12,
                                           color= 'rgba(0,0,0,1.0)'),
                                showarrow=False,))

    for yrev, xrevd in zip(y_revenue, rev_labels):    
        annotations.append(dict(xref='x2', yref='y2', y = xrevd, x = yrev,
                                xanchor = 'right',
                                text= '$'+'{:,}'.format(yrev),
                                font = dict(family='Arial',size=12,
                                           color= 'rgba(0,0,0,1.0)'),
                                showarrow=False,))

    for yvis, xvisd in zip(y_visits, vis_labels):
        annotations.append(dict(xref='x3', yref='y3', y = xvisd, x = yvis,
                                xanchor = 'right',
                                text='{:,}'.format(yvis) + ' Visits',
                                font = dict(family='Arial',size=12,
                                           color= 'rgba(0,0,0,1.0)'),
                                showarrow=False,))




    for yrep, xrepd in zip(y_repeats, rep_labels):    
        annotations.append(dict(xref='x4', yref='y4', y = xrepd , x = yrep,
                                xanchor = 'right',
                                text=str(yrep) + ' Distinct Groups',
                                font = dict(family='Arial',size=12,
                                           color= 'rgba(0,0,0,1.0)'),
                                showarrow=False,))

    layout['annotations'] = annotations

    fig = tools.make_subplots(rows=4, cols=1, specs=[[{}],[{}],[{}],[{}]], shared_xaxes=False,
                             shared_yaxes=True)

    fig.append_trace(trace0, 1, 1)
    fig.append_trace(trace1, 2, 1)
    fig.append_trace(trace2, 3, 1)
    fig.append_trace(trace3, 4, 1)

    fig['layout'].update(layout)
    fig['layout'].update(
        barmode = 'group',
        bargroupgap=0,
        bargap=0.3,
        autosize = True
    )
    TwoYearComparison = py.iplot(fig, filename='monarch-groupsales-twoyear-comparison')
    return TwoYearComparison
    #py.iplot(fig, filename='monarch-groupsales-twoyear-comparison')

def twoYearHistogram(data2015, data2016, parameter, scale, index, parameter2, metricName):

    graph = pd.DataFrame(columns= ['data2015','data2016'])
    if index == 'yes':
        
        graph['data2015'] = data2015[parameter]/ data2015[parameter2]
        graph['data2016'] = data2016[parameter]/ data2016[parameter2]
        
    else:
        graph['data2015'] = data2015[parameter]
        graph['data2016'] = data2016[parameter]
        
    
    if scale == 'yes':
        startpoint = graph['data2015'].min()
        endpoint = graph['data2015'].quantile(.9)
        binsize = ((endpoint - startpoint) / 20)
        autobin = False

        trace2015 = go.Histogram(
                x = graph['data2015'].as_matrix(),
                opacity = 0.75,
                name = '2014-2015 ' + metricName,
                autobinx = autobin,
                xbins=dict(
                    start= startpoint,
                    end = endpoint,
                    size = binsize),
                marker= dict(
                color= 'fuchsia',
                line= dict(
                    color='grey',
                    width=0
                )))

        trace2016 = go.Histogram(
                x = graph['data2016'].as_matrix(),
                opacity = 0.70,
                name = '2015-2016 ' + metricName,
                autobinx = autobin,
                xbins=dict(
                    start= startpoint,
                    end = endpoint,
                    size = binsize),
                marker = dict(
                color = 'rgb(255,217,102)',
                line = dict(
                    color='grey',
                    width=0)
                ))
    else:
        autobin = True
        trace2015 = go.Histogram(
            x = graph['data2015'].as_matrix(),
            opacity = 0.75,
            name = '2014-2015 ' + metricName,
            autobinx = autobin,
            marker= dict(
            color= 'fuchsia',
            line= dict(
                color='grey',
                width=0
            )))

        trace2016 = go.Histogram(
            x = graph['data2016'].as_matrix(),
            opacity = 0.70,
            name = '2015-2016 ' + metricName,
            autobinx = autobin,
            marker = dict(
            color = 'rgb(255,217,102)',
            line = dict(
                color='grey',
                width=0)
            ))

    data = [trace2015, trace2016]
    layout = go.Layout(
        title = 'Distribution of ' + metricName + ' in Group Sales Visits',
        xaxis= dict(
            title= metricName),
            yaxis=dict(
            title= 'Count'),
        barmode='overlay',
        bargap=0.25,
        bargroupgap=0.3)
            

    fig = go.Figure(data=data, layout=layout)

    histogram = py.iplot(fig, filename=parameter + '-histogram')
    return histogram


def scatterMatrix(data, parameters):
    matrix = data[parameters].scatter_matrix(filename='scatter-matrix', world_readable=True)
    return matrix

def twoYearScatter(data2015x,data2015y, data2016x, data2016y, ymetric, xmetric):
    trace2015 = go.Scatter(
        x = data2015x,
        y = data2015y,
        name = '2014-2015 Groups',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgba(255,182, 193, .9)',
            line = dict(
                width = 2,
                color = 'rgb(0,0,0)')
        )
    )

    trace2016 = go.Scatter(
        x = data2016x,
        y = data2016y,
        name = '2015-2016 Groups',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgba(152,0, 0, .8)',
            line = dict(
                width = 2,
                color = 'rgb(0,0,0)')
        )
    )

    data = [trace2015, trace2016]

    layout = dict(title = 'Group Sales Opportunities - ' + ymetric + ' versus ' + xmetric,
                 yaxis = dict( title = ymetric, zeroline = False),
                 xaxis = dict(title = xmetric, zeroline = False))

    fig = dict(data=data, layout=layout)
    scatterplot = py.iplot(fig, filename=ymetric + '-' + xmetric + 'scatter')
    return scatterplot


def Scatter3D():
    trace2015 = go.Scatter3d(
        x = clean_opps_2014to2015['Duration'],
        y = clean_opps_2014to2015['SkierDays']/clean_opps_2014to2015['Duration'],
        z = clean_opps_2014to2015['Revenue'],
        text = clean_opps_2014to2015['GroupName'],
        mode = 'markers',
        marker = dict(
            size=12,
            color = 'rgba(255,182, 193, .9)',
            line=dict(
                color = 'rgba(0, 0, 0, 0.14)',
                width = 0.5),
            opacity = 0.8
        )
    )

    trace2016 = go.Scatter3d(
        x = clean_opps_2015to2016['Duration'],
        y = clean_opps_2015to2016['SkierDays']/clean_opps_2015to2016['Duration'],
        z = clean_opps_2015to2016['Revenue'],
        text = clean_opps_2015to2016['GroupName'],
        mode = 'markers',
        marker = dict(
            size=12,
            color = 'rgba(152,0, 0, .8)',
            line=dict(
                color = 'rgba(0, 0, 0, 0.14)',
                width = 0.5),
            opacity = 0.8
        )
    )
    traceRepeats = go.Scatter3d(
        x = RepeatGroups_Instances_df['Duration'],
        y = RepeatGroups_Instances_df['SkierDays']/RepeatGroups_Instances_df['Duration'],
        z = RepeatGroups_Instances_df['Revenue'],
        text = RepeatGroups_Instances_df['AccountName'],
        mode = 'markers',
        marker = dict(
            size=12,
            color = 'rgba(175,69, 194, .8)',
            line=dict(
                color = 'rgba(0, 0, 0, 0.14)',
                width = 0.5),
            opacity = 0.8
        )
    )

    data = [trace2015, trace2016, traceRepeats]
    layout = go.Layout(
        title = 'Group Sales Opportunities - Visit Duration, Group Size, and Revenue',
        scene= go.Scene(
            xaxis=dict(
                title = 'Visit Duration',
                autorange=False,
                range=[0, 6]  # set axis range
            ),
            yaxis=dict(
                title = 'Group Size',
            ),
            zaxis=dict(
                title = 'Revenue',
            )),
        margin = dict(
            l=0,
            r=0,
            b=50,
            t=50))

    fig = go.Figure(data=data, layout=layout)
    Scatter_3D = py.iplot(fig, filename='Opportunities-3d-scatter')
    return Scatter_3D

def packagesBoxplot(yparam,xparam):
    data = [
        {
            'y': rentals_only[yparam],
            'x': rentals_only[xparam],
            'name': 'Lift Tickets and Rentals',
            'marker': {
                'color': '#3D9970'
            },
            'boxmean': False,
            'orientation': 'h',
            "type": "box",
        },
        {
            'y': lessons_only[yparam],
            'x': lessons_only[xparam],
            'name': 'Lift Tickets and Lessons',
            'marker': {
                'color': '#FF4136'
            },
            'boxmean': False,
            'orientation': 'h',
            "type": "box",
        },
        {
            'y': liftickets_only[yparam],
            'x': liftickets_only[xparam],
            'name': 'Lift Tickets Only',
            'marker': {
                'color': '#FF851B'
            },
            'boxmean': False,
            'orientation': 'h',
            "type": "box",
        },
        {
            'y': lessons_rentals[yparam],
            'x': lessons_rentals[xparam],
            'name': 'Lift Tickets,Lessons and Rentals',
            'marker': {
                'color': '#CD0074'
            },
            'boxmean': False,
            'orientation': 'h',
            "type": "box",
        }
    ]


    layout = {
        'title': 'Group Sales ' + xparam + ' by Package Type and ' + yparam + ' : 2014-2016',
        'xaxis': {
            'title': xparam,
            'autorange' : True,
            #'range' : [0, 13000]
        },
        'yaxis':{
            'title': yparam,
            'autorange' : True,
            #'range' : [0,6],
            'zeroline': False,
        },
        'boxmode': 'group',
    }
    fig = go.Figure(data=data, layout=layout)
    package_boxplot = py.iplot(fig, validate=False, 
                        filename='Group Sales ' + xparam + ' by Package Type and ' + yparam + ' : 2014-2016')
    return package_boxplot

def packageScatter(xparam, yparam):
    traceTLR = go.Scatter(
        x = lessons_rentals[xparam],
        y = lessons_rentals[yparam],
        text = lessons_rentals['GroupName'],
        name = 'Lift Tickets, Lessons and Rentals',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = '#CD0074',
            line = dict(
                width = 2,
            )
        )
    )

    traceT = go.Scatter(
        x = liftickets_only[xparam],
        y = liftickets_only[yparam],
        text = liftickets_only['GroupName'],
        name = 'Lift Tickets Only',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = '#FF851B',
            line = dict(
                width = 2,
            )
        )
    )

    traceTL = go.Scatter(
        x = lessons_only[xparam],
        y = lessons_only[yparam],
        text = lessons_only['GroupName'],
        name = 'Lift Tickets and Lessons',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = '#FF4136',
            line = dict(
                width = 2,
            )
        )
    )

    traceTR = go.Scatter(
        x = rentals_only[xparam],
        y = rentals_only[yparam],
        text = rentals_only['GroupName'],
        name = 'Lift Tickets and Rentals',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = '#3D9970',
            line = dict(
                width = 2,
            )
        )
    )

    data = [traceTLR, traceT, traceTL, traceTR]

    layout = dict(title = xparam + ' and ' + yparam + 'by Package Type 2014-2016',
                 yaxis = dict(zeroline = False, title = yparam),
                 xaxis = dict(zeroline = False, title = xparam))

    fig = dict(data=data, layout=layout)
    package_scatter = py.iplot(fig, filename = 'package-scatter-'+xparam+yparam)
    return package_scatter

def topGroups(data, season):
    trace0 = go.Bar(
        x=data['Revenue'],
        y=data['AccountName'],
        marker = dict(
            color='rgba(50,171,96,0.6)',
            line = dict(
                color='rgba(50,171,96,1.0)',
                width=1
                )
            ),
        name= 'Revenue',    
        orientation = 'h')

    trace1 = go.Scatter(
        x=data['SkierDays'],
        y=data['AccountName'],
        mode='lines+markers',
        line=dict(
            color='rgb(128,0,128)'),
        name='Skier Days'
        )

    layout = dict(
        title='Revenue and Skier Days from Group Sales Accounts Over $10k, ' + season,
        yaxis1=dict(
            showgrid=False,
            showline=False,
            showticklabels=True,
            domain=[0,0.85]),
        yaxis2=dict(
            showgrid=False,
            showline=True,
            showticklabels=False,
            linecolor='rgba(102,102,102,0.8)',
            linewidth=2,
            domain=[0,0.85]),
        xaxis1=dict(
            zeroline = False,
            showline=False,
            showticklabels=True,
            showgrid=True,
            domain=[0,0.53],),
        xaxis2=dict(
            zeroline=False,
            showline=False,
            showticklabels= True,
            showgrid=True,
            domain=[0.58,1],
            side='top',
            dtick = 250
            ),
        legend=dict(
            x=0.029,
            y=1.038,
            font=dict(
                size=10,
            ),
        ),
        margin=dict(
            l=400,
            r=0,
            t=70,
            b=70,
        ),
        width=1000,
        height=600,
        paper_bgcolor='rgb(248,248,255)',
        plot_bgcolor='rgb(248,248,255)'
    )

    annotations = []
    rev = np.round(data['Revenue'], decimals=2)
    skier = data['SkierDays']

    #Adding Labels
    for revs, skiers, accounts in zip(rev, skier, data['AccountName']):
        #labeling the scatter skier count
        annotations.append(dict(xref='x2', yref='y2', y=accounts,
                               x=skiers-140, text='{:,}'.format(skiers) + ' Skier Days',
                               font=dict(family='Arial', size=12,
                                        color='rgb(128,0,128)'),
                               showarrow=False,))
        #labeling the bar revenue
        annotations.append(dict(xref='x1', yref='y1', y=accounts,
                               x=revs + 300, text='                       $'+ '{:,}'.format(revs),
                               font=dict(family='Arial', size=12,
                                        color='rgb(50,171,96)'),
                               showarrow=False,))
        
    layout['annotations'] = annotations
    fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                           shared_yaxes= False, vertical_spacing=0.001)

    fig.append_trace(trace0, 1, 1)
    fig.append_trace(trace1, 1, 2)
    fig['layout'].update(layout)

    Over_10k_Groups = py.iplot(fig, filename='Over_10k_Groups-'+ season)
    return Over_10k_Groups

def industryMonth(data, parameter):
    trace0 = go.Bar(
        x=data[data.Month== 'November'].sort_values(by= parameter)['Industry'],
        y=data[data.Month== 'November'].sort_values(by= parameter)[parameter],
        name='November 2014')

    trace1 = go.Bar(
        x=data[data.Month== 'December'].sort_values(by= parameter)['Industry'],
        y=data[data.Month== 'December'].sort_values(by= parameter)[parameter],
        name='December 2014')
            
    trace2 = go.Bar(
        x=data[data.Month== 'January'].sort_values(by= parameter)['Industry'],
        y=data[data.Month== 'January'].sort_values(by= parameter)[parameter],
        name='January 2015')
                
    trace3 = go.Bar(
        x=data[data.Month== 'February'].sort_values(by= parameter)['Industry'],
        y=data[data.Month== 'February'].sort_values(by= parameter)[parameter],
        name='February 2015')
        
                    
    trace4 = go.Bar(
        x=data[data.Month== 'March'].sort_values(by= parameter)['Industry'],
        y=data[data.Month== 'March'].sort_values(by= parameter)[parameter],
        name='March 2015')
                        
    trace5 = go.Bar(
        x=data[data.Month== 'April'].sort_values(by= parameter)['Industry'],
        y=data[data.Month== 'April'].sort_values(by= parameter)[parameter],
        name='April 2015')

    data = [trace1, trace2, trace3, trace4, trace5]
    layout = go.Layout(
        barmode='stack',
        title = 'Group Sales Industry '+parameter + ' by Month; 2014-2015',
        xaxis=dict(
            title='Industry'
        ),
        yaxis=dict(
            title= parameter))
    fig = go.Figure(data=data, layout=layout)
    industry_month = py.iplot(fig, filename = 'industry-revenue')
    return industry_month

def timeseries_revenue_skierdays(data,season):
    timeseries_data = data.groupby('ArrivalDate').sum()
    timeseries_data = timeseries_data.reset_index()

    trace1 = go.Scatter(
        x=timeseries_data['ArrivalDate'],
        y=timeseries_data['SkierDays'],
        name = 'Skier Days',
        yaxis='y1'
        )
    trace2 = go.Scatter(
        x=timeseries_data['ArrivalDate'],
        y=timeseries_data['Revenue'],
        name = 'Gross Revenue',
        yaxis='y2')

    data = [trace1, trace2]
    layout = go.Layout(
        title = 'Total Daily Skier Count and Revenue, Group Sales ' +season,
        yaxis=dict(
            title='Skier Days',
        ),
        yaxis2=dict(
            title= 'Revenue',
            titlefont=dict(
                color='rgb(148,103,189)'
            ),
            tickfont=dict(
                color='rgb(148,103,189)'
            ),
            overlaying='y',
            side='right'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    timeseries = py.iplot(fig, filename='SkierCount-Revenue-Timeseries-' + season)
    return timeseries

def bookingTimeseries(data, season):
    booking_timeries_data = data.groupby('DateBooked').size()
    booking_timeries_data = booking_timeries_data.reset_index()

    trace1 = go.Scatter(
        x=booking_timeries_data['DateBooked'],
        y=booking_timeries_data[0],
        name = 'Number of Trips Booked per Day',
        yaxis='y1'
    )


    data = [trace1]
    layout = go.Layout(
        title='Number of Trips Booked per Day, Group Sales' + season,
        yaxis=dict(
            title='Number of Trips Booked',
        ),
    )

    fig = go.Figure(data=data, layout=layout)
    booking_timeseries = py.iplot(fig, filename='DateBooked-Timeseries-2016')
    return booking_timeseries

def industry_dataframe_view(data):
    view = data.groupby('Industry').sum()
    view_multiday = data[data.Duration > 1]
    view_multiday_grouped = view_multiday.groupby('Industry').sum()

    view['MultiDayGroupCount'] = valueCount(view_multiday, 'Industry')
    view['TotalGroupCount'] = valueCount(data, 'Industry')
    
    return view

def state_dataframe_view(data):
    state_view = data.groupby('State').sum()
    state_multiday = data[data.Duration > 1]

    state_view['MultiDayGroupCount'] = valueCount(state_multiday, 'State')
    state_view['TotalGroupCount'] = valueCount(data, 'State')
    
    return state_view

def packages_statistics():
    columns = ['Metric',
             """Lift Tickets Only: """ + str(len(liftickets_only)) + """ Groups""" ,
             """Lift Tickets + Rentals: """ + str(len(rentals_only)) + """ Groups""",
             """Lift Tickets + Lessons: """ +str(len(lessons_only)) + """ Groups""",
             """Lift Tickets + Rentals + Lessons: """ + str(len(lessons_rentals)) + """ Groups""",
             "Statistic"]
    stats =[['Skier Count', liftickets_only['SkierCount'].sum(), rentals_only['SkierCount'].sum(), lessons_only['SkierCount'].sum(), lessons_rentals['SkierCount'].sum(), 'Sum'],
          ['Skier Count', liftickets_only['SkierCount'].mean(), rentals_only['SkierCount'].mean(), lessons_only['SkierCount'].mean(), lessons_rentals['SkierCount'].mean(), 'Mean'],
          ['Skier Count', liftickets_only['SkierCount'].median(), rentals_only['SkierCount'].median(), lessons_only['SkierCount'].median(), lessons_rentals['SkierCount'].median(), 'Median'],
          ['Skier Days', liftickets_only['SkierDays'].sum(), rentals_only['SkierDays'].sum(), lessons_only['SkierDays'].sum(), lessons_rentals['SkierDays'].sum(), 'Sum'],
          ['Skier Days', liftickets_only['SkierDays'].mean(), rentals_only['SkierDays'].mean(), lessons_only['SkierDays'].mean(), lessons_rentals['SkierDays'].mean(), 'Mean'],
          ['Skier Days', liftickets_only['SkierDays'].median(), rentals_only['SkierDays'].median(), lessons_only['SkierDays'].median(), lessons_rentals['SkierDays'].median(), 'Median'],
          ['Group Size Per Day', (liftickets_only['SkierDays']/liftickets_only['Duration']).sum(), (rentals_only['SkierDays']/rentals_only['Duration']).sum(), (lessons_only['SkierDays']/lessons_only['Duration']).sum(), (lessons_rentals['SkierDays']/lessons_rentals['Duration']).sum(), 'Sum'],
          ['Group Size Per Day', (liftickets_only['SkierDays']/liftickets_only['Duration']).mean(), (rentals_only['SkierDays']/rentals_only['Duration']).mean(), (lessons_only['SkierDays']/lessons_only['Duration']).mean(), (lessons_rentals['SkierDays']/lessons_rentals['Duration']).mean(), 'Mean'],
          ['Group Size Per Day', (liftickets_only['SkierDays']/liftickets_only['Duration']).median(), (rentals_only['SkierDays']/rentals_only['Duration']).median(), (lessons_only['SkierDays']/lessons_only['Duration']).median(), (lessons_rentals['SkierDays']/lessons_rentals['Duration']).median(), 'Median'],
          ['Trip Duration', liftickets_only['Duration'].sum(), rentals_only['Duration'].sum(), lessons_only['Duration'].sum(), lessons_rentals['Duration'].sum(), 'Sum'],
          ['Trip Duration', liftickets_only['Duration'].mean(), rentals_only['Duration'].mean(), lessons_only['Duration'].mean(), lessons_rentals['Duration'].mean(), 'Mean'],
          ['Trip Duration', liftickets_only['Duration'].median(), rentals_only['Duration'].median(), lessons_only['Duration'].median(), lessons_rentals['Duration'].median(), 'Median'],
          ['Revenue', liftickets_only['Revenue'].sum(), rentals_only['Revenue'].sum(), lessons_only['Revenue'].sum(), lessons_rentals['Revenue'].sum(), 'Sum'],
          ['Revenue', liftickets_only['Revenue'].mean(), rentals_only['Revenue'].mean(), lessons_only['Revenue'].mean(), lessons_rentals['Revenue'].mean(), 'Mean'],
          ['Revenue', liftickets_only['Revenue'].median(), rentals_only['Revenue'].median(), lessons_only['Revenue'].median(), lessons_rentals['Revenue'].median(), 'Median'],
          ['Revenue per Person', (liftickets_only['Revenue']/liftickets_only['Revenue']).sum(), (rentals_only['Revenue']/rentals_only['Revenue']).sum(), (lessons_only['Revenue']/lessons_only['Revenue']).sum(), (lessons_rentals['Revenue']/lessons_rentals['Revenue']).sum(), 'Sum'],
          ['Revenue per Person', (liftickets_only['Revenue']/liftickets_only['Revenue']).mean(), (rentals_only['Revenue']/rentals_only['Revenue']).mean(), (lessons_only['Revenue']/lessons_only['Revenue']).mean(), (lessons_rentals['Revenue']/lessons_rentals['Revenue']).mean(), 'Mean'],
          ['Revenue per Person', (liftickets_only['Revenue']/liftickets_only['Revenue']).median(), (rentals_only['Revenue']/rentals_only['Revenue']).median(), (lessons_only['Revenue']/lessons_only['Revenue']).median(), (lessons_rentals['Revenue']/lessons_rentals['Revenue']).median(), 'Median'],
         ]

    packages_table = pd.DataFrame(stats,columns=columns)

    packages_table = pd.DataFrame(stats,columns=columns)

    packages_table_pivot = packages_table.pivot('Metric', 'Statistic').round(2).transpose()

    return packages_table_pivot

def annual_comparison_stats():
    columns = ['Metric', '2014-2015', '2015-2016', 'Difference', 'Statistic']
    stats_overall = [['Skier Days', SkierDays2015_Sum, SkierDays2016_Sum, abs(SkierDays2015_Sum -SkierDays2016_Sum), 'Sum'],
            ['Skier Days', SkierDays2015_Mean, SkierDays2016_Mean, abs(SkierDays2015_Mean -SkierDays2016_Mean), 'Mean'],
            ['Skier Days', SkierDays2015_Median, SkierDays2016_Median, abs(SkierDays2015_Median -SkierDays2016_Median), 'Median'],
            ['Skier Count', SkierCount2015_Sum, SkierCount2016_Sum, abs(SkierCount2015_Sum -SkierCount2016_Sum), 'Sum'],
            ['Skier Count', SkierCount2015_Mean, SkierCount2016_Mean, abs(SkierCount2015_Mean -SkierCount2016_Mean), 'Mean'],
            ['Skier Count', SkierCount2015_Median, SkierCount2016_Median, abs(SkierCount2015_Median -SkierCount2016_Median), 'Median'],
            ['Group Size Per Day', GroupSize2015_Sum, GroupSize2016_Sum, abs(GroupSize2015_Sum - GroupSize2016_Sum), 'Sum'],
            ['Group Size Per Day', GroupSize2015_Mean, GroupSize2016_Mean, abs(GroupSize2015_Mean - GroupSize2016_Mean), 'Mean'],
            ['Group Size Per Day', GroupSize2015_Median, GroupSize2016_Median, abs(GroupSize2015_Median - GroupSize2016_Median), 'Median'],
            ['Trip Duration', Duration2015_Sum, Duration2016_Sum, abs(Duration2015_Sum -Duration2016_Sum), 'Sum'],
            ['Trip Duration', Duration2015_Mean, Duration2016_Mean, abs(Duration2015_Mean -Duration2016_Mean), 'Mean'],
            ['Trip Duration', Duration2015_Median, Duration2016_Median, abs(Duration2015_Median -Duration2016_Median), 'Median'],
            ['Revenue', Revenue2015_Sum, Revenue2016_Sum, abs(Revenue2015_Sum -Revenue2016_Sum), 'Sum'],
            ['Revenue', Revenue2015_Mean, Revenue2016_Mean, abs(Revenue2015_Mean -Revenue2016_Mean), 'Mean'],
            ['Revenue', Revenue2015_Median, Revenue2016_Median, abs(Revenue2015_Median -Revenue2016_Median), 'Median'],
            ['Revenue per Person', Revenue_per_person2015_Sum, Revenue_per_person2016_Sum, abs(Revenue_per_person2015_Sum -Revenue_per_person2016_Sum), 'Sum'],
            ['Revenue per Person', Revenue_per_person2015_Mean, Revenue_per_person2016_Mean, abs(Revenue_per_person2015_Mean -Revenue_per_person2016_Mean), 'Mean'],
            ['Revenue per Person', Revenue_per_person2015_Median, Revenue_per_person2016_Median, abs(Revenue_per_person2015_Median -Revenue_per_person2016_Median), 'Median']]

    stats_table = pd.DataFrame(stats_overall,columns=columns)

    stats_table_pivot = stats_table.pivot('Metric', 'Statistic').round(2)
    return stats_table_pivot

In [151]:
#Data Wrangling Section
params = Opportunities_DataFrame('SalesforceAPI.txt')
params.querySalesforce()
params.buildDataFrame()
opps = params.Opportunities_df

clean_opps, clean_opps_2015to2016, clean_opps_2014to2015 = spliceYears(opps)
account_count_alltime = valueCount(clean_opps, 'AccountName')
account_count_2014to2015 = valueCount(clean_opps_2014to2015, 'AccountName')
account_count_2015to2016 = valueCount(clean_opps_2015to2016, 'AccountName')
Visits_2014to2015 = groupVisits(clean_opps_2014to2015, '2014to2015')
Visits_2015to2016 = groupVisits(clean_opps_2015to2016, '2015to2016')

Skierdays_2014to2015 = list_of_Column_Values(clean_opps_2014to2015, 'SkierDays')
Skierdays_2015to2016 = list_of_Column_Values(clean_opps_2015to2016, 'SkierDays')
Revenue_2014to2015 = list_of_Column_Values(clean_opps_2014to2015, 'Revenue')
Revenue_2015to2016 = list_of_Column_Values(clean_opps_2015to2016, 'Revenue')

AlltimeVisits_2015to2016groups = clean_opps_2015to2016[['VisitCount_AllTIme', 'VisitCount_2015to2016']]
RepeatGroups_Instances_df, RepeatGroups_Instances_grouped = RepeatGroups(clean_opps_2015to2016, clean_opps_2014to2015)

SkierDays2015_Sum = sum_Column(clean_opps_2014to2015, 'SkierDays')
SkierDays2016_Sum = sum_Column(clean_opps_2015to2016, 'SkierDays')
SkierCount2015_Sum = sum_Column(clean_opps_2014to2015, 'SkierCount')
SkierCount2016_Sum = sum_Column(clean_opps_2015to2016, 'SkierCount')
Duration2015_Sum = sum_Column(clean_opps_2014to2015, 'Duration')
Duration2016_Sum = sum_Column(clean_opps_2015to2016, 'Duration')
GroupSize2015_Sum = sum_adjusted_SkierDays(clean_opps_2014to2015)
GroupSize2016_Sum = sum_adjusted_SkierDays(clean_opps_2015to2016)
Revenue2015_Sum = sum_Column(clean_opps_2014to2015, 'Revenue')
Revenue2016_Sum = sum_Column(clean_opps_2015to2016, 'Revenue')
Revenue_per_person2015_Sum = Revenue2015_Sum/GroupSize2015_Sum
Revenue_per_person2016_Sum = Revenue2016_Sum/GroupSize2016_Sum

SkierDays2015_Mean = mean_Column(clean_opps_2014to2015, 'SkierDays')
SkierDays2016_Mean = mean_Column(clean_opps_2015to2016, 'SkierDays')
SkierCount2015_Mean = mean_Column(clean_opps_2014to2015, 'SkierCount')
SkierCount2016_Mean = mean_Column(clean_opps_2015to2016, 'SkierCount')
Duration2015_Mean = mean_Column(clean_opps_2014to2015, 'Duration')
Duration2016_Mean = mean_Column(clean_opps_2015to2016, 'Duration')
GroupSize2015_Mean = mean_adjusted_SkierDays(clean_opps_2014to2015)
GroupSize2016_Mean = mean_adjusted_SkierDays(clean_opps_2015to2016)
Revenue2015_Mean = mean_Column(clean_opps_2014to2015, 'Revenue')
Revenue2016_Mean = mean_Column(clean_opps_2015to2016, 'Revenue')
Revenue_per_person2015_Mean = Revenue2015_Mean/GroupSize2015_Mean
Revenue_per_person2016_Mean = Revenue2016_Mean/GroupSize2016_Mean

SkierDays2015_Median = median_Column(clean_opps_2014to2015, 'SkierDays')
SkierDays2016_Median = median_Column(clean_opps_2015to2016, 'SkierDays')
SkierCount2015_Median = median_Column(clean_opps_2014to2015, 'SkierCount')
SkierCount2016_Median = median_Column(clean_opps_2015to2016, 'SkierCount')
Duration2015_Median = median_Column(clean_opps_2014to2015, 'Duration')
Duration2016_Median = median_Column(clean_opps_2015to2016, 'Duration')
GroupSize2015_Median = mean_adjusted_SkierDays(clean_opps_2014to2015)
GroupSize2016_Median = mean_adjusted_SkierDays(clean_opps_2015to2016)
Revenue2015_Median = median_Column(clean_opps_2014to2015, 'Revenue')
Revenue2016_Median = median_Column(clean_opps_2015to2016, 'Revenue')
Revenue_per_person2015_Median = Revenue2015_Median/GroupSize2015_Median
Revenue_per_person2016_Median = Revenue2016_Median/GroupSize2016_Median

    
#Segment Lift Ticket, Rentals, and Lesson Combination Packages
rentals_only = clean_opps[(clean_opps.Rentals==True) & (clean_opps.Lesson==False)]
lessons_only = clean_opps[(clean_opps.Lesson==True) & (clean_opps.Rentals==False)]
liftickets_only = clean_opps[(clean_opps.Lesson==False) & (clean_opps.Rentals==False)]
lessons_rentals = clean_opps[(clean_opps.Lesson==True) & (clean_opps.Rentals==True)]

industries2016 = category_stats(clean_opps_2015to2016, 'Industry')
industries2016_html = generate_html_table(industries2016)

timeseries_2014to2015 = timeseries(clean_opps_2014to2015)
timeseries_2015to2016 = timeseries(clean_opps_2015to2016)
timeseries_2014to2015 = timeseries(clean_opps)

Over10k_Groups2 = topAccounts(clean_opps)


/Users/AnthonyAbercrombie/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [152]:
#Create Graphs
twoYearOverview = twoYearOverview()

#Histograms
revenueHistogram = twoYearHistogram(clean_opps_2014to2015, clean_opps_2015to2016, 'Revenue', 'yes','no','na', 'Revenue')
skierdaysHistogram = twoYearHistogram(clean_opps_2014to2015, clean_opps_2015to2016, 'SkierDays', 'yes', 'no','na', 'Skier Days')
durationHistogram = twoYearHistogram(clean_opps_2014to2015, clean_opps_2015to2016, 'Duration', 'no', 'no', 'na', 'Trip Duration')
skiercountHistogram = twoYearHistogram(clean_opps_2014to2015, clean_opps_2015to2016, 'SkierCount', 'yes', 'no', 'na', 'Skier Count')
avgDailySkiersHistogram = twoYearHistogram(clean_opps_2014to2015, clean_opps_2015to2016, 'SkierDays', 'yes', 'yes', 'Duration', 'Average Daily Skiers')
revenuePerPersonHistogram = twoYearHistogram(clean_opps_2014to2015, clean_opps_2015to2016, 'Revenue', 'yes', 'yes', 'SkierCount', 'Revenue per Person')

#Correlation Matrix
scattermatrix_2years = scatterMatrix(clean_opps, ['Revenue', 'SkierDays','Duration','SkierCount'])

#Scatter Plots
skiercount_duration_Scatter = twoYearScatter(clean_opps_2014to2015['Duration'],clean_opps_2014to2015['SkierCount'], clean_opps_2015to2016['Duration'], clean_opps_2015to2016['SkierCount'], 'Skier Count', 'Trip Duration')
skierdays_revenue_Scatter =  twoYearScatter(clean_opps_2014to2015['SkierDays'],clean_opps_2014to2015['Revenue'], clean_opps_2015to2016['SkierDays'], clean_opps_2015to2016['Revenue'], 'Revenue', 'Skier Days')
Scatter_3D = Scatter3D()

#Package Boxplots
packages_duration_revenue_Box = packagesBoxplot('Duration','Revenue')
packages_duration_skierdays_Box = packagesBoxplot('Duration','SkierDays')
packages_state_revenue_Box = packagesBoxplot('State','Revenue')
packages_transport_revenue_Box = packagesBoxplot('Transportation','Revenue')

#Package Scatterplots
packages_skierdays_revenue_Scatter = packageScatter('SkierDays', 'Revenue')
packages_duration_skiercount_Scatter = packageScatter('Duration', 'SkierCount')

#Top Groups over $10k 2014 - 2016
topGroups2014to2016 = topGroups(Over10k_Groups2, '2014 - 2016')

#Industry by Month Charts
industry_revenue_2015to2016 = industryMonth(clean_opps_2015to2016, 'Revenue')
industry_revenue_2014to2015 = industryMonth(clean_opps_2014to2015, 'Revenue')
industry_skierdays_2015to2016 = industryMonth(clean_opps_2015to2016, 'SkierDays')
industry_skierdays_2014to2015 = industryMonth(clean_opps_2014to2015, 'SkierDays')

#Timeseries of Revenue and Skier Days
timeseries_revenue_skierdays_2014to2016 = timeseries_revenue_skierdays(clean_opps,'2014 - 2016')
timeseries_revenue_skierdays_2014to2015 = timeseries_revenue_skierdays(clean_opps_2014to2015,'2014 - 2015')
timeseries_revenue_skierdays_2015to2016 = timeseries_revenue_skierdays(clean_opps_2015to2016,'2015 - 2016')

#Timeseries of Number of Group Reservations per Day
bookingTimeseries = bookingTimeseries(clean_opps,'2014 - 2016')

#Table Views of Industry Statistics
Industries2016 = industry_dataframe_view(clean_opps_2015to2016)
Industries2015 = industry_dataframe_view(clean_opps_2014to2015)
Industries_2014to2016 = industry_dataframe_view(clean_opps)

#Table Views of State Statistics
States2016 = state_dataframe_view(clean_opps_2015to2016)
States2015 = state_dataframe_view(clean_opps_2014to2015)
States_2014to2016 = state_dataframe_view(clean_opps)

#Table View of Package Statistics
package_pivot_table = packages_statistics()

#Table View of Statistics Comparing the Last Two Ski Seasons
annual_comparison_pivot_table = annual_comparison_stats()


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [155]:
#WRITE HTML REPORT
html_string = '''

    <html>
        <head>
            <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
            <style>body{ margin:0 100; background:whitesmoke; }</style>
        </head>
        <body>
            <h1>Monarch Group Sales 2015-2016 Report</h1>
            <h4>Report created by Anthony Abercrombie; April 2016.</h4>

            <h2>Abstract</h2>
            <p>The objectives of this report are to summarize Monarch Group Sales activity in the 2015 - 2016 season, measure performance in comparison to the 2014 - 2015 season, and explore patterns in Monarch's CRM data. The intention of performing this analysis is to define and clarify Group Sales' market share and provide a platform for growing Group Sales revenue, adding more skiers to Monarch's customer base, reducing churn, and improving the overall group experience at Monarch.</p>
            <p>The results of the report are organized by the general business questions they address. These are questions that management may be interested in for assessing performance and identifying opportunities for improvement.</p>
            <ol>
              <li>How did Monarch Group Sales perform this season in comparison to last season?</li>
              <li>Where are Monarch's groups coming from?</li>
              <li>When did Group Sales' skier attendance and revenue peak?</li>
              <li>When are Monarch's groups booking their reservations?</li>
              <li>Who are Monarch's groups and who are Monarch's highest value constituents?</li>
              <li>Who were Monarch's most valuable groups the last two seasons?</li>
              <li>How are Monarch's groups distributed along their performance indicators? What are the patterns in the data?</li>
              <li>Which packages are groups buying?</li>
            </ol>


            <h2>Methods</h2>
            <p>This report draws on data from Monarch's Salesforce CRM database. Monarch started using Salesforce to track group accounts and opportunities at the beginning of the 2014-2015 ski season, so only two years of data are available for analysis.</p>
            <p>Data was obtained by SQL querying Salesforce with the SOAP API. Figures presented in the report update automatically after the query is renewed.</p>
            <p>Data cleaning and wrangling was performed using Python's pandas and numpy libraries in an IPython notebook IDE. Interactive data visualizations were coded in JSON format and powered by Plotly's Python API, which renders JSONs into D3.JS graphics.</p>

            <p>Source code can be found on GitHub at https://github.com/AAbercrombie0492/Monarch_Salesforce.git . Data is confidential and Monarch's API login credentials are not publicly available.</p>


            <h2>Results:</h2>

            <h3>1.How did Monarch Group Sales perform this season in comparison to last season?</h3>
            <iframe width="1000" height="500" frameborder="0" seamless="seamless" scrolling="no" src=https://plot.ly/~AAbercrombie0492/34.embed?width=800&height=550"> </iframe>
            <p><b>Figure 1: Graphical overview of Skier Days, Revenue, Visits, and Groups in the 2014-2015 season and the 2015-2016 season.</b></p>

            <table border="1" class="dataframe table">  <thead>    <tr>      <th></th>      <th colspan="3" halign="left">2014-2015</th>      <th colspan="3" halign="left">2015-2016</th>      <th colspan="3" halign="left">Difference</th>    </tr>    <tr>      <th></th>      <th>Mean</th>      <th>Median</th>      <th>Sum</th>      <th>Mean</th>      <th>Median</th>      <th>Sum</th>      <th>Mean</th>      <th>Median</th>      <th>Sum</th>    </tr>    <tr>      <th>Metric</th>      <th></th>      <th></th>      <th></th>      <th></th>      <th></th>      <th></th>      <th></th>      <th></th>      <th></th>    </tr> </thead>  <tbody>    <tr>      <th>Group Size Per Day</th>      <td>22.88</td>      <td>16.00</td>      <td>6542.50</td>      <td>20.34</td>      <td>15.00</td>      <td>5613.04</td>      <td>2.54</td>      <td>1.00</td>     <td>929.46</td>    </tr>    <tr>      <th>Revenue</th>     <td>1741.91</td>      <td>1267.00</td>      <td>501670.15</td>      <td>1691.50</td>     <td>1265.00</td>      <td>465162.93</td>      <td>50.41</td>      <td>2.00</td>     <td>36507.22</td>    </tr>    <tr>      <th>Revenue per Person</th>      <td>76.15</td>      <td>79.19</td>      <td>76.68</td>      <td>83.17</td>      <td>84.33</td>      <td>82.87</td>      <td>7.03</td>      <td>5.15</td>      <td>6.19</td>    </tr>    <tr>      <th>Skier Count</th>      <td>23.47</td>      <td>17.00</td>      <td>7371.00</td>      <td>23.39</td>      <td>16.00</td>      <td>7227.00</td>      <td>0.09</td>      <td>1.00</td>      <td>144.00</td>    </tr>    <tr>      <th>Skier Days</th>      <td>41.56</td>      <td>30.00</td>      <td>11929.00</td>      <td>37.57</td>      <td>27.50</td>      <td>10370.00</td>      <td>3.99</td>      <td>2.50</td>      <td>1559.00</td>    </tr>    <tr>      <th>Trip Duration</th>      <td>1.84</td>      <td>2.00</td>      <td>577.00</td>      <td>2.04</td>      <td>2.00</td>      <td>629.00</td>      <td>0.20</td>      <td>0.00</td>      <td>52.00</td>    </tr>  </tbody></table>
            <p><b>Figure 2: Summary statistics on Group Sales performance indicators across the last two ski seasons</b></p>
            <p>
              Group Size per Day is an adjusted measure of Skier Days that accounts for variability in group member attendance over the course of a group's visit. For example, a 20 person group may have a 3 day reservation at Monarch, but only 15 of those 20 skiers may ski all 3 days. Group Size per Day = Skier Days / Trip Duration, so it essentially measures a group's average Skier Days per day.</p> 

            <p>The number of unique groups and visits from 2015-2016 is not far off from 2015-2015's stats. In 2015-2016, Group Sales brought in 1,642 fewer Skier Days than 2014-2015. The lower Skier Day count factors into a revenue lag of $39,775.15 behind 2014-2015.</p>

            <p>In the 2015-2016 season, 40% of our groups were repeat groups from 2014-2015. Of the 309 group visits Group Sales hosted in the 2015-2016, 54% of those group visits can be attributed to repeat groups from both years.</p>



            <h3>2. Where are Monarch's groups coming from?</h3>
            <iframe width="100%" height="520" frameborder="0" src="https://amarinusha.cartodb.com/viz/01ada270-f9da-11e5-9eb8-0e31c9be1b51/embed_map" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>
            <p><b>Figure 3: Geographic representation of Group Sales groups by skier days and season classification.</b></p>

            <iframe width="100%" height="520" frameborder="0" src="https://amarinusha.cartodb.com/viz/929f40d6-f9d5-11e5-ad6f-0e31c9be1b51/embed_map" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>
            <p><b>Figure 4: Geographic representation of Group Sales groups by revenue and season classification. </b></p>

            <iframe width="100%" height="520" frameborder="0" src="https://amarinusha.cartodb.com/viz/69fdcbd8-f9db-11e5-b567-0ecd1babdde5/embed_map" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>
            <p><b>Figure 5: Cluster explorer that allows a user to quickly quantify the number of groups that originate from a given region.</b></p>

            <iframe width="100%" height="520" frameborder="0" src="https://amarinusha.cartodb.com/viz/c0561516-f9d7-11e5-97c7-0ecfd53eb7d3/embed_map" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>
            <p><b>Figure 6: Heat map that depicts the regional density of groups.</b></p>

            <p>
              Lookup addresses for each group are associated with the group leader's billing address, not the organization's billing address. This broadens the geographic range of groups that purchase tickets through wholesalers.</p>

            <p>Groups from 2014-2015 seem to visually indicate a broader range of geographic origins than 2015-2016 groups.</p>


            <h3>3. When did Group Sales' skier attendance and revenue peak?</h3>
            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/79.embed?width=800&height=550"></iframe>
            <p><b>Figure 7: Time series of revenue and skier days over the course of the 2015-2016 season. Revenue and skier days from multi-day groups factor into the group's arrival date.</b></p>


            <iframe width="100%" height="520" frameborder="0" src="https://amarinusha.cartodb.com/viz/4df768d8-f9d9-11e5-9cd2-0e31c9be1b51/embed_map" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>
            <p><b>Figure 8: Video map of group density by arrival date and point of origin from 2014 - 2016.</b></p>

            <p>
              Peak periods in Group Sales traffic occur between Christmas and New Year's, mid-January (Martin Luther King Jr. Weekend), the week following Valentine's day, and Spring Break.</p>

            <h3>4. When are Monarch's groups booking their reservations?</h3>
            <iframe width="900" height="800" frameborder="0" scrolling="no" src="https://plot.ly/~AAbercrombie0492/83.embed"></iframe>
            <p><b>Figure 9: Number of trips booked per day from 2014-2016. </b></p>
            <p> 12/08/2016 and 01/24/2016 show unusually high reservation counts. </p>

            <h3>5. Who are Monarch's groups and who are Monarch's highest value constituents?</h3>
            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/14.embed?width=800&height=550"></iframe>
            <p><b>Figure 10: Gross revenue by industry, drilled down by month of arrival date.</b></p>

            <h4>Group Sales Industry Stats, 2014 - 2016: </h4>
            <table border="1" class="dataframe table">  <thead>    <tr style="text-align: right;">      <th>Industry</th>      <th>Trip Duration</th>      <th>Lesson</th>      <th>Rentals</th>      <th>Revenue</th>      <th>Skier Count</th>      <th>Skier Days</th>    </tr>  </thead>  <tbody>    <tr>      <td>Business</td>      <td>19</td>      <td>4</td>      <td>5</td>      <td>16751.0</td>      <td>620</td>      <td>648</td>    </tr>    <tr>      <td>Camp</td>      <td>9</td>      <td>2</td>      <td>2</td>      <td>5823.0</td>      <td>69</td>      <td>162</td>    </tr>    <tr>      <td>Charity</td>      <td>2</td>      <td>1</td>      <td>0</td>      <td>138.0</td>      <td>17</td>      <td>7</td>    </tr>    <tr>      <td>Church</td>      <td>244</td>      <td>71</td>      <td>117</td>      <td>257176.0</td>      <td>3008</td>      <td>4531</td>    </tr>    <tr>      <td>Club</td>      <td>31</td>      <td>10</td>      <td>16</td>      <td>20521.0</td>      <td>451</td>      <td>531</td>    </tr>    <tr>      <td>Colorado School</td>      <td>159</td>      <td>71</td>      <td>82</td>      <td>115721.0</td>      <td>3597</td>      <td>2957</td>    </tr>    <tr>      <td>Family</td>      <td>305</td>      <td>59</td>      <td>95</td>      <td>188198.0</td>      <td>1710</td>      <td>3845</td>    </tr>    <tr>      <td>Healthcare</td>      <td>3</td>      <td>2</td>      <td>2</td>      <td>2205.0</td>      <td>71</td>      <td>58</td>    </tr>    <tr>      <td>Higher Education</td>      <td>79</td>      <td>21</td>      <td>35</td>      <td>56999.0</td>      <td>850</td>      <td>1020</td>    </tr>    <tr>      <td>Military</td>      <td>11</td>      <td>2</td>      <td>2</td>      <td>44172.0</td>      <td>115</td>      <td>1517</td>    </tr>    <tr>      <td>Organization</td>      <td>20</td>      <td>9</td>      <td>9</td>      <td>27622.0</td>      <td>382</td>      <td>608</td>    </tr>    <tr>      <td>Other</td>      <td>6</td>      <td>1</td>      <td>2</td>      <td>2842.0</td>      <td>39</td>      <td>62</td>    </tr>    <tr>      <td>Out of State School</td>      <td>13</td>      <td>3</td>      <td>7</td>      <td>9212.0</td>      <td>148</td>      <td>154</td>    </tr>    <tr>      <td>Wholesaler</td>      <td>290</td>      <td>81</td>      <td>43</td>      <td>258454.9</td>      <td>3481</td>      <td>7539</td>    </tr>  </tbody></table>
          <p><b>Figure 11: Performance indicator totals by industry, 2014-2016 </b></p>
          <p>
            Most group categories generate their largest monthly revenues in March. The exceptions are Colorado Schools and Higher Education groups, which bring the most of their business in January. </p>
  
            <h3>6. Who were Monarch's most valuable groups the last two seasons?</h3>
            <iframe width="1000" height="520" frameborder="0" seamless="seamless"scrolling="no"         src=https://plot.ly/~AAbercrombie0492/73.embed?width=1000
            &height=520"></iframe>

            <p><b> Figure 12: Revenue and skier days from accounts valued at over $10,000 over the last two ski seasons.</b></p>
            <p>
              6 of the 12 groups shown above are wholesalers. Revenue and Skier Days are representative of the last 2 ski seasons. </p>

            <h3>7. How are Monarch's groups distributed along their performance indicators? What are the patterns in the data?</h3>
            <iframe width="900" height="800" frameborder="0" scrolling="no" src="https://plot.ly/~AAbercrombie0492/81.embed"></iframe>
            <p><b>Figure 13: Correlation matrix of performance indicators; skier days, skier count, revenue, and trip duration</b></p>

            <p>
              Skier Days, Skier Count, and Trip Duration distributions are all highly left skewed. I will need to examine the normality of Revenue's residuals in order to determine whether a linear or logistic regression model is appropriate. </p>

            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/77.embed?width=800&height=550"></iframe>

            <p><b>Figure 14: Histogram distribution of group revenues, indicating the number of groups from each year that fall within a set of revenue brackets.</b></p>

            <p> Overall, the 2014-2015 season shows a greater number of high revenue groups than the 2015-2016 season. Visually, it is ambigous whether this distribution in normal enough for linear regression. Residuals need to be examined.</p>

            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/45.embed?width=800&height=550"></iframe>
            <p><b>Figure 15: Histogram distribution of group skier days from each year.</b></p>

            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/57.embed?width=800&height=550"></iframe>
            <p><b>Figure 16: Histogram distribution of group visit durations from each year.</b></p>

            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/55.embed?width=800&height=550"></iframe>
            <p><b>Figure 17: Histogram distribution of group skier counts from each year.</b></p>

            <p>The difference in Revenue and Skier Days between 2014-2015 and 2015-2016 is visually apparent, but there does not seem to be major differences in the Visit Duration and Skier Count distributions that would explain these differences. The following graph visualizes Group Size per Day, which depicts a group's Skier Days divided by their Trip Duration.</p>

            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/67.embed?width=800&height=550"></iframe>
            <p><b>Figure 18: Histogram distribution of group size per day from each year's groups.</b></p>

            
            <p>
              Even though Skier Counts and Trip Durations from both years are comparable, it seems that 2015-2016 groups tended to have lower Group Sizes per Day. This means that more participants within the 2015-2016 groups chose to not ski the full duration of their group's reservation. 
            </p>
           

            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/61.embed?width=800&height=550"></iframe>
            <p><b> Figure 19: Scatterplot of group visit duration and skier count for each year.</b></p>

            <p>More outlier groups with high skier counts are observed in the 3 day visit category. </p>

            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/63.embed?width=800&height=550"></iframe>
            <p><b>Figure 20: Scatterplot of group skier days and revenue for each year. </b></p>

            <p>Beyond the cluster of groups below $4k and 100 Skier Days, most of the high yield outlier groups are from Wholesalers. 2014-2015 outliers appear to be higher in Revenue and more numerous than 2015-2016 outliers.</p>

            <iframe width="800" height="700" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/65.embed?width=800
            &height=700"></iframe>
            <p><b>Figure 21: 3D Scatterplot of Visit Duration, Group Size/Skier Count, and Revenue. Pink dots are 2014-2015 groups, red dots are 2015-2016 groups, and purple dots are 2015-2016 groups that Group Sales retained from 2014-2015.</b></p>

            <p>Most of 2015-2016's outlier groups with high revenues are repeats from the previous year.</p>

            <h3>8. Which packages are groups buying?</h3>
            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"src=https://plot.ly/~AAbercrombie0492/69.embed?width=800&height=550"></iframe>
            <p><b>Figure 22: Box plot illustrating the ranges in revenue generated by each package type for each visit duration. </b></p>
            <p>When a group makes a reservation online, they check a box to signify whether anyone in their group will need rentals or lessons. The data available from Monarch's Salesforce accounting is limited in that it does allow us to examine the number of indivuals within each group that purchase lessons or rent gear. </p>

            <iframe width="800" height="520" frameborder="0" seamless="seamless" scrolling="no"             src=https://plot.ly/~AAbercrombie0492/71.embed?width=800&height=550"></iframe>
            <p><b>Figure 23: Scatter plot of skier days and revenue, grouped by package type.</b></p>

            <p>Lift Ticket + Lessons groups show a wide range in Revenue yield. This may be explained by the observation that many Wholesaler groups purchase lift tickets and lessons with Monarch, but rent gear elsewhere.</p>


            <h4> Group Statistics 2014-2016 Grouped by Package Type. </h4>
            <table border="1" class="dataframe table">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Metric</th>      <th>Group Size Per Day</th>      <th>Revenue</th>      <th>Revenue per Person</th>      <th>Skier Count</th>      <th>Skier Days</th>      <th>Trip Duration</th>    </tr>  </thead>  <tbody>    <tr>      <th rowspan="3" valign="top">Lift Tickets Only: 158 Groups</th>      <th>Mean</th>      <td>16.90</td>      <td>1218.89</td>      <td>1</td>      <td>18.27</td>      <td>40.23</td>      <td>1.78</td>    </tr>    <tr>      <th>Median</th>      <td>13.00</td>      <td>630.00</td>      <td>1</td>      <td>14.00</td>      <td>20.00</td>      <td>1.00</td>    </tr>    <tr>      <th>Sum</th>      <td>2197.12</td>      <td>162112.00</td>      <td>133</td>      <td>2667.00</td>      <td>5391.00</td>      <td>260.00</td>    </tr>    <tr>      <th rowspan="3" valign="top">Lift Tickets + Rentals: 138 Groups</th>      <th>Mean</th>      <td>17.17</td>      <td>1439.02</td>      <td>1</td>      <td>18.14</td>      <td>29.17</td>      <td>1.77</td>    </tr>    <tr>      <th>Median</th>      <td>14.00</td>      <td>1065.50</td>      <td>1</td>      <td>15.00</td>      <td>22.00</td>      <td>1.00</td>    </tr>    <tr>      <th>Sum</th>      <td>2060.08</td>      <td>172682.68</td>      <td>120</td>      <td>2503.00</td>      <td>3500.00</td>      <td>244.00</td>    </tr>    <tr>      <th rowspan="3" valign="top">Lift Tickets + Lessons: 57 Groups</th>      <th>Mean</th>      <td>36.23</td>      <td>2588.40</td>      <td>1</td>      <td>34.26</td>      <td>82.70</td>      <td>2.32</td>    </tr>    <tr>      <th>Median</th>      <td>25.00</td>      <td>1725.53</td>      <td>1</td>      <td>25.00</td>      <td>52.00</td>      <td>2.00</td>    </tr>    <tr>      <th>Sum</th>      <td>1920.25</td>      <td>139773.56</td>      <td>54</td>      <td>1953.00</td>      <td>4383.00</td>      <td>132.00</td>    </tr>    <tr>      <th rowspan="3" valign="top">Lift Tickets + Rentals + Lessons: 285 Groups</th>      <th>Mean</th>      <td>23.09</td>      <td>2049.08</td>      <td>1</td>      <td>26.51</td>      <td>39.92</td>      <td>2.01</td>    </tr>    <tr>      <th>Median</th>      <td>17.00</td>      <td>1706.50</td>      <td>1</td>      <td>20.00</td>      <td>32.00</td>      <td>2.00</td>    </tr>    <tr>      <th>Sum</th>      <td>6049.08</td>      <td>536859.84</td>      <td>261</td>      <td>7554.00</td>      <td>10459.00</td>      <td>573.00</td>    </tr>  </tbody></table>
            <p><b>Figure 24: Group Statistics 2014-2016 grouped by package type.</b></p>

            <p>Lift Tickets + Rentals + Lessons groups brought the most Revenue, Skiers, and Group Days to Monarch from 2014-2016. Lift Ticket + Lessons groups had the highest mean and median figures across all performance metrics.</p>

            <h3>Discussion</h3>

            <h4>1. How did Monarch Group Sales perform this season in comparison to last season?</h4>

            <p>The number of group visits and the the number of unique groups is more or less equivalent for the 2014-2015 and 2015-2016 seasons (Figures 1, 2, 16, and 17). Lower revenue and skier day figures from this year can be explained by a lower average group size per day and smaller Wholesaler groups in comparison to last year. (Figures 2, 18)</p>

            <p>In the 2015-2016 season, Group Sales' skier count (anticipated group size pre-arrival) was higher on average (difference of 0.18 skiers per group) and in total (difference of 39 skiers throughout the season).The same goes for trip duration, which on average was 0.2 days greater and over the course of the season was greater by 56 visit days. Revenue per Skier was greater in 2015-2016 by $7.72. (Figures 1 & 2)</p>  

            <p>Why are group participants cutting ski days off their visit? Since 40% of 2015-2016 groups are returning groups from the previous year, it makes since that participants that have skied Monarch before would be less interested in skiing multiple days in a row. How can Monarch maintain the interest of returning participants? If these participants are spending part of their trips away from the mountain, which local activites and attractions are they drawn to? Are there opportunities for local business partnerships and referral programs, which could retain some lost revenue, that haven't been implemented yet?</p> 

            <h4>2. Where are Monarch's groups coming from?</h4>

            <p>In the 2015-2016 season, 85% of Group Sales <b>revenue </b> came from <b>Colorado (50.61%), Texas (19.07%), and Kansas (15.11%) </b> groups (See 2015-2016 analysis spreadsheets). 87% of Group Sales skier days are attributed to Colorado (58.99%), Texas (15.46%), and Kansas (12.79%). </p>
            <p>The following states rank in the top 5 for highest <b>revenue per group</b>:</p>
            <ol>
              <li><b>Missouri</b> - $4,113.50 per group </li>
              <li><b>Oklahoma</b> - $3,051 per group </li>
              <li><b>Arkansas</b> - $2,626.67 per group </li>
              <li><b>Louisiana</b> - $2,536 per group </li>
              <li><b>Texas</b> - $2,213.76 per group </li>
            </ol>
            <p>The following states rank in the top 5 for highest <b>revenue per skier</b>:</p>
            <ol>
              <li><b>Alabama</b> - $84.50 per skier</li>
              <li><b>Arkansas</b> - $70.36 per skier</li>
              <li><b>Tennessee</b> - $59.58 per skier</li>
              <li><b>Missouri</b> - $58.35 per skier</li>
              <li><b>Texas</b> - $55.27 per skier</li>
            </ol>
            <p>Visual exploration of the maps presented in Figures 3 - 6 may assist in planning future sales, marketing, and business development trips in the midwest. Here a few recommendations for further investigation: </p>
            <ul>
              <li>Kansas City, MO area, 12 groups from 2014-2016</li>
              <li>Oklahoma City, OK - 7 groups from 2014-2016</li>
              <li>Siloam Springs and Fort Smith, AR - 4 groups from 2014 - 2016. </li>
              <li>Shreveport, LA - 1 group from 2014-2016</li>
              <li>Baton Rouge/New Orleans, LA - 3 groups from 2014-2016</li>
              <li>Dallas/Fort Worth, TX  - 22 groups from 2014-2016</li>
              <Albuquerque, NM - 8 groups from 2014-2016</li>
            </ul>

            <h4>3. When did Group Sales' skier attendance and revenue peak?</h4>

            <p>In the 2015-2016 season, peak periods in Group Sales traffic occured between Christmas and New Year's, mid-January (Martin Luther King Jr. Weekend), the week following Valentine's day, and Spring Break. The skier day and revenue amounts that correspond to each date are the aggregate sums of all groups that started their trips on that date. High revenue groups often reserve multi-day trips, so it is important to keep in mind that their entire revenue and skier day amounts are aggregated into their first day for the purpose of visualization. (Figures 7 and 8) </p>


            <h4>4. When are Monarch's groups booking their reservations?</h4>

            <p>Figure 9 shows the number of group reservations that were booked per day from 2014-2016. These dates correspond with the date that a Group Sales employee opened an opportunity in Salesforce. Reservations are made online before Salesforce data entry, so it is possible that there are mismatches between date reserved online and date opened.</p>
            <p>Groups likely plan and budget their trips far in advance of the date they make a reservation online. When Group Sales sends out surveys to this year's groups, the survey will ask the group leader when their organization started planning and budgeting for the trip. This information, combined with customer segmentation and cluster analysis, can help Monarch can get a better idea of when groups start to consider ski trips and when to market for optimal conversion.</p> 

            <h4>5. Who are Monarch's groups and who are Monarch's highest value constituents?</h3>

            <p> To reference the following statistics, see the Group Sales spreadsheet analysis</p>

            <p> In the <b>2015-2016 season</b>, 84.57% of Monarch's Group Sales revenue came from: </p>



            <ol>
              <li><b>Wholesalers</b> (26.65%)</li>
              <li><b>Churches</b> (23.67%)</li>
              <li><b>Families</b> (20.92%)</li>
              <li><b>Colorado Schools</b> (13.33%)</li>
            </ol>

            <p> In the <b>2014-2015 season</b>, 85.36% of Monarch's Group Sales revenue came from: </p>

            <ol>
              <li><b>Wholesalers</b> (32.82%)</li>
              <li><b>Families</b> (18.87%)</li>
              <li><b>Churches</b> (18.76%)</li>
              <li><b>Colorado Schools</b> (14.90%)</li>
            </ol>

            <p>Churches, and to a lesser extent families, became more significant sources of revenue in the 2015-2016 season. <p>

            <p>Figure 10 shows that for most group types, the majority of revenue comes in the month of March. The only two exceptions are Colorado Schools and Higher Education, which both bring the majority of their revenue in January. This may support the case for increasing marketing and promotional efforts to schools in January, or decreasing the number of School Group Rate days in January.</p>

            <p>The following group types rank in the top 5 for highest <b>revenue per group</b> in the 2015-2016 season:</p>

            <ol>
              <li><b>Wholesalers</b> - $2,029.05 per group</li>
              <li><b>Churches</b> - $1,965.47</li>
              <li><b>Organizations</b> - $1,836.22</li>
              <li><b>Families</b> - $1592.75</li>
              <li><b>Clubs</b> - $1,431.86</li>
            </ol>

            <p>The following group types rank in the top 5 for highest <b>revenue per skier</b> in the 2015-2016 season:</p>

            <ol>
              <li><b>Higher Education</b> - $59.74 per Skier</li>
              <li><b>Churches</b> - $56.39 per Skier</li>
              <li><b>Out of State Schools</b> - $55.50 per Skier</li>
              <li><b>Organizations</b> - $49.93 per Skier</li>
              <li><b>Families</b> - $49.57 per Skier</li>
            </ol>

            <p>Over the course of the last two seasons,Church groups had the highest number of groups that rented gear (117 group visits that included rentals). Wholesaler groups had the highest number of groups that purchased lessons (81 groups). Families spent the most time at Monarch the last two seasons at a total of 305 visit days.</p>

            <p>While the Wholesaler, School, Church, and Family classifications are useful, there is a limited amount of insight that can be deduced from the data with a priori classifications. Statistical and Machine Learning methods including cluster analysis, hierarchical classification, and scoring models can be used to detect more nuanced groupings and associations in our customer base. Further analysis that includes demographic, geographic, and behavioral characteristics of Monarch's groups can enable more precise, intelligent marketing.</p>

            <h4>6. Who were Monarch's most valuable groups the last two seasons? </h3>

            <p>Needless to say, Tommy Query of White Diamond Ski Tours is THE MAN. The 12 groups listed in Figure 12 are accountable for $295,724.47 over the last two seasons, or 31% of gross revenue. These same groups are also accountable for 8,058 skier days, or 36% of the total skier days from 2014-2016.</p>

            <h4>7. How are Monarch's groups distributed along their performance indicators? What are the patterns in the data?</h3>

            <p> Skier days, skier count, and trip duration all skew highly to the left. A large proportion of our groups are small (10 - 20 people) and stay for a day or two at most.Averages for these indicators are affected by few outlier groups with large number of skiers that generate high revenue.</p>

            <p>Figure 21 shows that many of the high value outlier groups from 2016 are repeats from 2015, which is great! Instead of focusing on attaining more groups and group visits each season, Monarch Group Sales may consider focusing on retaining more high value groups and up-selling its loyal customer base. </p>

            <h4>8. Which packages are groups buying?</h3>

            <p>Lift Tickets + Rentals + Lessons groups brought the most Revenue, Skiers, and Group Days to Monarch from 2014-2016. Lift Ticket + Lessons groups had the highest mean and median figures across all performance metrics.</p>

            <p>In order to do a more thorough analysis of package purchasing through group sales, Group Sales workbooks need to be mined, cleaned, wrangled, and merged so that purchasing decisions can be examined at the scale of the individual. This can be done either by querying REZ  group accounts or writing a script that parses all of the workbooks on Monarch's shared network drive.</p>

            <h3>Conclusion</h3>
            <p> There is still a lot of work that needs to be done to analyze and better understand Monarch's customer base. Examining the distributions of Group Sales data indicates that Group Sales revenue is driven by fewer, larger groups that are often tied to Wholesalers. Although Group Sales only accounts for 4% of Monarch's annual revenue on average, Group Sales introduces large quantities of beginner skiers and Monarch first-timers to the mountain, expanding Monarch's potential market share. </p>
            <p> According to the 2014-2015 Group Sales report, only 6% of 2014-2015 groups were repeats from the previous season. In the 2015-2016 season, 40% of the groups were repeats from the last season. This surge in customer loyalty calls for a sincere commendation to Hope Rheingans, CJ Heartland, and all of the Group Sales hosts. These individuals have promoted a highly positive group experience on a daily basis and have fostered valuable relationships with many of Group Sales' returning group leaders.</p>
            <p>Moving forward with a new POS system, CRM practices, and software changes in sight, Monarch group sales will have more time to focus on big wins. With less time spent on managing customer orders and balances, Sales and Marketing staff will hopefully be able to spend more time understanding its customers, increasing retention while reducing churn, acquiring higher value business, and keeping returning customers engaged with Monarch and the local economy.</p>

        </body>
    </html>
'''

report = open('/Users/AnthonyAbercrombie/projects/Monarch_Salesforce/report_test.html','w')
report.write(html_string)
report.close()